In [32]:
import zarr
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

In [33]:
def build_mission_root(dataset_folder, mission):
    base = Path(dataset_folder) / mission / "data"
    mission_root = {}

    for d in sorted(base.iterdir()):
        if d.is_dir() and (d / ".zgroup").exists():
            try:
                g = zarr.open_group(store=zarr.storage.LocalStore(str(d)), mode="r")
                mission_root[d.name] = g
            except Exception as e:
                print(f"[skip] {d.name}: {e}")

    return mission_root

dataset_folder = Path("~/grand_tour_dataset/missions").expanduser()
mission = "2024-10-01-11-47-44"

mission_root = build_mission_root(dataset_folder, mission)

In [34]:
sensor = "alphasense_front_center"
sensor_timestamp = mission_root[sensor]["timestamp"]
print([float(x) for x in sensor_timestamp[0:3]])

[1727776101.9499006, 1727776102.0498998, 1727776102.1498995]


In [114]:
sensors = [
    "anymal_state_odometry",
    "anymal_state_state_estimator",
    "anymal_imu",
    "anymal_state_actuator",
    "anymal_command_twist",
    "hdr_front",
    "hdr_left",
    "hdr_right"
]

In [101]:
cmd_tstmp = mission_root["anymal_command_twist"]["timestamp"]
print([f"{float(x)}" for x in cmd_tstmp[0:20]])

['1727776107.42612', '1727776107.4762018', '1727776107.5262368', '1727776107.5766277', '1727776107.6261902', '1727776107.676212', '1727776107.7262874', '1727776107.776655', '1727776107.826147', '1727776107.8762813', '1727776107.9262016', '1727776107.976376', '1727776108.0262923', '1727776108.0762396', '1727776108.126283', '1727776108.1761966', '1727776108.2261224', '1727776108.2762027', '1727776108.3261607', '1727776108.3765135']


In [103]:
est_tstmp = mission_root["anymal_state_state_estimator"]["timestamp"]
print([f"{float(x)}" for x in est_tstmp[145:165]])

['1727776102.3312173', '1727776102.333793', '1727776102.3361952', '1727776102.338843', '1727776102.341199', '1727776102.34374', '1727776102.3462842', '1727776102.3487394', '1727776102.3511715', '1727776102.3540738', '1727776102.3561873', '1727776102.3588126', '1727776102.3613245', '1727776102.3639505', '1727776102.3664956', '1727776102.368871', '1727776102.371332', '1727776102.3739412', '1727776102.3764946', '1727776102.3787956']


In [116]:
def get_hz(x):
    timestamps = np.array(x)
    diffs = np.diff(timestamps)
    avg_dt = np.mean(diffs)
    frequency_hz = 1 / avg_dt
    return f"{frequency_hz:.2f}"
    #print(f"Average frequency:  Hz")

In [118]:
for s in sensors:
    timestamps =  mission_root[s]["timestamp"]
    hz = get_hz(timestamps)
    print(f"{s} {hz} Hz {timestamps.shape}")

anymal_state_odometry 19.96 Hz (8663,)
anymal_state_state_estimator 399.96 Hz (173589,)
anymal_imu 400.01 Hz (173529,)
anymal_state_actuator 399.94 Hz (173318,)
anymal_command_twist 15.74 Hz (5739,)
hdr_front 10.00 Hz (4226,)
hdr_left 10.00 Hz (4226,)
hdr_right 10.00 Hz (4226,)


In [ ]:
#mohammad version 

for s in sensors:
    timestamps =  mission_root[s]["timestamp"]
    start = np.min(timestamps)
    end = np.max(timestamps)
    total_samples = timestamps.shape[0]
    print(f"{s} {total_samples/(end-start)}")


anymal_state_odometry 19.96429418762061
anymal_state_state_estimator 399.9586178280574
anymal_imu 400.01067416518663
anymal_state_actuator 399.9405558502917
anymal_command_twist 15.742669996289393
hdr_front 10.002366875194083
hdr_left 10.002366875194083
hdr_right 10.002366875194083


In [ ]:

idx_start = 0
idx_range = 3
for s in sensors:
    sensor_timestamp = mission_root[s]["timestamp"]
    sensor_seqid = mission_root[s]["sequence_id"]
    print(f"{s} {sensor_seqid.shape}")
    #print(f"{s} {[float(x) for x in sensor_timestamp[idx_start:idx_start+idx_range]]} {[int(x) for x in sensor_seqid[idx_start:idx_start+idx_range]]}")
    #print()

anymal_state_odometry (8663,)
anymal_state_state_estimator (173589,)
anymal_imu (173529,)
anymal_state_actuator (173318,)
anymal_command_twist (5739,)
hdr_front (4226,)
hdr_left (4226,)
hdr_right (4226,)


In [108]:
ref_ts = mission_root["anymal_command_twist"]["timestamp"][0]

# store aligned timestamps
aligned_timestamps = {}

for sensor in sensors:
    ts = np.array(mission_root[sensor]["timestamp"])
    
    # find the first index where timestamp >= ref_ts
    idx = np.searchsorted(ts, ref_ts, side="left")
    
    # take the next 5 timestamps starting from that index
    aligned_timestamps[sensor] = ts[idx:idx+5]

# print the result
for k, v in aligned_timestamps.items():
    print(f"{k}: {list(map(float,v))}")

anymal_state_odometry: [1727776107.4586549, 1727776107.5087557, 1727776107.5586689, 1727776107.608674, 1727776107.6587613]
anymal_state_state_estimator: [1727776107.4263504, 1727776107.4287694, 1727776107.4315941, 1727776107.4336796, 1727776107.4363296]
anymal_imu: [1727776107.42761, 1727776107.4300508, 1727776107.4325721, 1727776107.4350438, 1727776107.4376154]
anymal_state_actuator: [1727776107.4261851, 1727776107.428562, 1727776107.4312088, 1727776107.4335272, 1727776107.4360466]
anymal_command_twist: [1727776107.42612, 1727776107.4762018, 1727776107.5262368, 1727776107.5766277, 1727776107.6261902]
hdr_front: [1727776107.511734, 1727776107.611735, 1727776107.711735, 1727776107.8117359, 1727776107.9117343]
hdr_left: [1727776107.511734, 1727776107.611735, 1727776107.711735, 1727776107.8117359, 1727776107.9117343]
hdr_right: [1727776107.511734, 1727776107.611735, 1727776107.711735, 1727776107.8117359, 1727776107.9117343]


In [104]:

target = 1727776107.42612

timestamps = np.array(mission_root["hdr_front"]["timestamp"])

lower_bound_mask = timestamps >= target
upper_bound_mask = timestamps <= target + 1

combined_mask = lower_bound_mask & upper_bound_mask
found_indices = np.where(combined_mask)[0]

for i in found_indices:
    print(f"FOUND: index={i}, timestamp={timestamps[i]}")


# around the 50th image is when the first command comes so the robot starts moving.

FOUND: index=55, timestamp=1727776107.511734
FOUND: index=56, timestamp=1727776107.611735
FOUND: index=57, timestamp=1727776107.711735
FOUND: index=58, timestamp=1727776107.8117359
FOUND: index=59, timestamp=1727776107.9117343
FOUND: index=60, timestamp=1727776108.0117345
FOUND: index=61, timestamp=1727776108.1117346
FOUND: index=62, timestamp=1727776108.2117345
FOUND: index=63, timestamp=1727776108.311734
FOUND: index=64, timestamp=1727776108.4117339


In [60]:
sensor = mission_root["anymal_state_state_estimator"]
for f in list(sensor.keys()):
    print(f"{f} {sensor[f].shape}")


LF_FOOT_friction_coef (173589,)
LF_FOOT_restitution_coef (173589,)
LF_FOOT_contact (173589,)
joint_efforts (173589, 12)
joint_velocities (173589, 12)
joint_accelerations (173589, 12)
LF_FOOT_state (173589,)
joint_positions (173589, 12)
LF_FOOT_normal (173589, 3)
LH_FOOT_friction_coef (173589,)
LF_FOOT_wrench_force (173589, 3)
LH_FOOT_normal (173589, 3)
LH_FOOT_state (173589,)
LH_FOOT_restitution_coef (173589,)
LH_FOOT_wrench_force (173589, 3)
LH_FOOT_contact (173589,)
LF_FOOT_wrench_torque (173589, 3)
pose_pos (173589, 3)
RF_FOOT_friction_coef (173589,)
LH_FOOT_wrench_torque (173589, 3)
RF_FOOT_contact (173589,)
RF_FOOT_restitution_coef (173589,)
RF_FOOT_normal (173589, 3)
RF_FOOT_wrench_force (173589, 3)
RF_FOOT_wrench_torque (173589, 3)
RF_FOOT_state (173589,)
pose_orien (173589, 4)
RH_FOOT_normal (173589, 3)
RH_FOOT_contact (173589,)
RH_FOOT_friction_coef (173589,)
RH_FOOT_restitution_coef (173589,)
RH_FOOT_state (173589,)
RH_FOOT_wrench_torque (173589, 3)
RH_FOOT_wrench_force (1735

In [ ]:
sensor = mission_root["anymal_imu"]
imu_df = pd.DataFrame({
    "timestamp": imu["timestamp"][:],
    "sequence_id": imu["sequence_id"][:],
    "ang_vel_x": imu["ang_vel"][:, 0],
    "ang_vel_y": imu["ang_vel"][:, 1],
    "ang_vel_z": imu["ang_vel"][:, 2],
    "lin_acc_x": imu["lin_acc"][:, 0],
    "lin_acc_y": imu["lin_acc"][:, 1],
    "lin_acc_z": imu["lin_acc"][:, 2],
    "orien_w": imu["orien"][:, 0],
    "orien_x": imu["orien"][:, 1],
    "orien_y": imu["orien"][:, 2],
    "orien_z": imu["orien"][:, 3],
})

In [ ]:
tstmp = mission_root["hdr_front"]["timestamp"]
min_t = tstmp[135]-2
max_t = tstmp[146]+2 

In [56]:
imu_window = imu_df[(imu_df["timestamp"] >= min_t) & (imu_df["timestamp"] <= max_t)]

In [58]:
imu_window.to_csv("imu_window.csv", index=False)

In [ ]:


est = mission_root["anymal_state_state_estimator"]

# Core arrays
ts   = est["timestamp"][:].astype(np.float64)             # (N,)
seq  = est["sequence_id"][:].astype(np.int64)             # (N,)
q    = est["joint_positions"][:]                          # (N, 12)
qd   = est["joint_velocities"][:]                         # (N, 12)
qdd  = est["joint_accelerations"][:]                      # (N, 12)

# Base pose & twist
pose_pos   = est["pose_pos"][:]                           # (N, 3) -> x,y,z
pose_orien = est["pose_orien"][:]                         # (N, 4) -> quat (x,y,z,w)
twist_lin  = est["twist_lin"][:]                          # (N, 3) -> vx,vy,vz
twist_ang  = est["twist_ang"][:]                          # (N, 3) -> wx,wy,wz

# Per-leg contacts (assumed 0/1 or small int)
LFc = est["LF_FOOT_contact"][:]
LHc = est["LH_FOOT_contact"][:]
RFc = est["RF_FOOT_contact"][:]
RHc = est["RH_FOOT_contact"][:]

# Per-leg wrenches; use Z (normal load) as the main signal
LF_F = est["LF_FOOT_wrench_force"][:]                     # (N, 3)
LH_F = est["LH_FOOT_wrench_force"][:]
RF_F = est["RF_FOOT_wrench_force"][:]
RH_F = est["RH_FOOT_wrench_force"][:]

# ----- Helpers -----
def quat_to_euler_xyz(q):
    """
    q: (..., 4) quaternion in (x, y, z, w) convention.
    returns roll (x), pitch (y), yaw (z) in radians.
    """
    x, y, z, w = q[...,0], q[...,1], q[...,2], q[...,3]
    # roll (x-axis)
    t0 = +2.0*(w*x + y*z)
    t1 = +1.0 - 2.0*(x*x + y*y)
    roll = np.arctan2(t0, t1)
    # pitch (y-axis)
    t2 = +2.0*(w*y - z*x)
    t2 = np.clip(t2, -1.0, 1.0)
    pitch = np.arcsin(t2)
    # yaw (z-axis)
    t3 = +2.0*(w*z + x*y)
    t4 = +1.0 - 2.0*(y*y + z*z)
    yaw = np.arctan2(t3, t4)
    return roll, pitch, yaw

roll, pitch, yaw = quat_to_euler_xyz(pose_orien)

# ----- Build joint column names -----
J = q.shape[1]  # expect 12
pos_cols = [f"q_{j}"   for j in range(J)]
vel_cols = [f"qd_{j}"  for j in range(J)]
acc_cols = [f"qdd_{j}" for j in range(J)]

# ----- Assemble DataFrame -----
est_df = pd.DataFrame({
    "timestamp": ts,
    "sequence_id": seq,
    # base pose
    "base_x": pose_pos[:,0],
    "base_y": pose_pos[:,1],
    "base_z": pose_pos[:,2],
    # base orientation (Euler, radians)
    "roll":  roll,
    "pitch": pitch,
    "yaw":   yaw,
    # base twist
    "vx": twist_lin[:,0],
    "vy": twist_lin[:,1],
    "vz": twist_lin[:,2],
    "wx": twist_ang[:,0],
    "wy": twist_ang[:,1],
    "wz": twist_ang[:,2],
    # per-leg contacts
    "LF_contact": LFc.astype(np.int8),
    "LH_contact": LHc.astype(np.int8),
    "RF_contact": RFc.astype(np.int8),
    "RH_contact": RHc.astype(np.int8),
    # per-leg vertical GRF (normal loads)
    "LF_Fz": LF_F[:,2],
    "LH_Fz": LH_F[:,2],
    "RF_Fz": RF_F[:,2],
    "RH_Fz": RH_F[:,2],
})

# total normal load (sum of feet)
est_df["sum_Fz"] = est_df[["LF_Fz","LH_Fz","RF_Fz","RH_Fz"]].sum(axis=1)

# Add joint arrays (positions/velocities/accelerations)
est_df[pos_cols] = q
est_df[vel_cols] = qd
est_df[acc_cols] = qdd


In [85]:
est_window = est_df[(est_df["timestamp"] >= min_t) & (est_df["timestamp"] <= max_t)].reset_index(drop=True)

In [86]:
est_window.to_csv("est_window.csv", index=False)

In [93]:
#est_window["base_z"].max() - est_window["base_z"].min()
print(est_window["base_z"].max())
print(est_window["base_z"].min())
# base increases --> going up steps. so the data aligns

10.76544064783801
10.583691188782703


In [96]:
list(est_window["base_z"])

[10.583691188782703,
 10.584266224135105,
 10.584266224135105,
 10.585369047069056,
 10.586036541090547,
 10.58665193921596,
 10.587308728268702,
 10.587935225787264,
 10.588632452096071,
 10.58930615066442,
 10.59001623731632,
 10.59001623731632,
 10.59134856309494,
 10.591994819282883,
 10.592586126203695,
 10.59331370168147,
 10.59396477784473,
 10.594524853315857,
 10.595112210243697,
 10.595112210243697,
 10.596119538500496,
 10.596498594948525,
 10.596808286794154,
 10.597152809472531,
 10.597473525499284,
 10.597747072318846,
 10.597979501934196,
 10.59813973777862,
 10.59825202211792,
 10.598332016097213,
 10.598374554270814,
 10.598377275289115,
 10.59833818941098,
 10.59826010090884,
 10.59814163066402,
 10.59797496936859,
 10.597758526163343,
 10.597474958652079,
 10.597171830820022,
 10.596824033220875,
 10.596432283659647,
 10.596003132138183,
 10.595472393859563,
 10.59495163996038,
 10.594387504112099,
 10.59383962276353,
 10.59383962276353,
 10.59293404615894,
 10.59252

In [131]:
1/(0.005*4)

50.0